In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 

%matplotlib inline

In [2]:
import sys  
sys.path.insert(0, '..')

In [3]:
from tiling.read_tiles import TissueDataset, load_color_normalization_values

In [4]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
HDF5_FOLDER = '/media/nico/data/fourthbrain/project/training_CAMELYON16'
green_layer_only = False
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [6]:
train_data = TissueDataset(HDF5_FOLDER,  percentage=0.8, first_part=True, crop_size=256)
val_data = TissueDataset(HDF5_FOLDER, percentage=0.2, first_part=False, crop_size=256)

In [7]:
batch_size_neg=50
batch_size_pos=50
batches_per_train_epoch = 100
batches_per_val_epoch = 50
MAX_EPOCHS = 10

In [8]:
def history_to_pickle(history, filepath):
    with open(filepath, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
        
def pickle_to_history(filepath):
    history = pickle.load(open(filepath, "rb"))
    return history

In [9]:
def plot_history(history):
    # list all data in history
    print(history.keys())
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
# FEATURE EXTRACTOR MODEL

In [10]:
model = keras.applications.InceptionV3(input_shape=(256,256,3),
                                        include_top=True,
                                        classes = 1,
                                        classifier_activation="sigmoid",
                                        weights=None)


In [11]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [13]:
model_type = 'train_no_filters_rgb_normalized_level_3'
model_hdf5 = '/media/nico/data/fourthbrain/project/model_testing/model_{}_'.format(model_type) + datetime.now().strftime("%Y%m%d-%H%M%S") + '.hdf5'
filepath = '/media/nico/data/fourthbrain/project/model_testing/{}.pkl'.format(model_type)
log_dir = "/media/nico/data/fourthbrain/project/model_testing/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [14]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_hdf5, monitor='loss',verbose=1, save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, 
                           verbose=1, mode='max', min_lr=1e-5)

#all callbacks
callbacks = [tensorboard_callback, model_checkpoint, reduce_lr]  

In [15]:
now1 = datetime.now()
    
history = model.fit(x=train_data.generator(batch_size_neg, batch_size_pos, True, green_layer_only=False, color_normalization_file=None),
                    validation_data=val_data.generator(batch_size_neg, batch_size_pos, False, green_layer_only=False, color_normalization_file=None),
                    epochs=MAX_EPOCHS,
                    steps_per_epoch = batches_per_train_epoch,
                    validation_steps = batches_per_val_epoch,
                    callbacks=callbacks
                   )

dtme = now1.strftime('%m%d%Y_%H%M%S.pkl')
now2 = datetime.now()

Epoch 1/10
100/100 [==============================] - 48s 387ms/step - loss: 0.2684 - accuracy: 0.8922 - val_loss: 0.7563 - val_accuracy: 0.5000

Epoch 00001: loss improved from inf to 0.21550, saving model to /media/nico/data/fourthbrain/project/model_testing/model_train_no_filters_rgb_normalized_level_3_20210125-220204.hdf5
Epoch 2/10
100/100 [==============================] - 36s 364ms/step - loss: 0.1617 - accuracy: 0.9352 - val_loss: 0.6464 - val_accuracy: 0.5790

Epoch 00002: loss improved from 0.21550 to 0.16231, saving model to /media/nico/data/fourthbrain/project/model_testing/model_train_no_filters_rgb_normalized_level_3_20210125-220204.hdf5
Epoch 3/10
100/100 [==============================] - 36s 363ms/step - loss: 0.1252 - accuracy: 0.9525 - val_loss: 1.1537 - val_accuracy: 0.5038

Epoch 00003: loss improved from 0.16231 to 0.12469, saving model to /media/nico/data/fourthbrain/project/model_testing/model_train_no_filters_rgb_normalized_level_3_20210125-220204.hdf5
Epoch 4/

In [16]:
print('duration: {} - path: {}'.format(now2 - now1, filepath))
history_to_pickle(history, filepath)

duration: 0:06:34.626797 - path: /media/nico/data/fourthbrain/project/model_testing/train_no_filters_rgb_normalized_level_3.pkl


In [21]:
model.save('model_inceptionv3.hdf5')

In [22]:
# FEATURE MAP CONSTRUCTION

In [26]:
# Model layer
layer_output = model.layers[-1].output

# layer_outputs = [layer.output for layer in model.layers]


# feature_model = Model(inputs=model_googlenet.input, outputs=layer_outputs)
# feature_map_total = []
# for x in range(0,len(X_train)):
#   feature_map = feature_model.predict(X_train[x].reshape(1,256,256,3)) 
#   feature_map_total.append(feature_map)

In [28]:
layer_output

AttributeError: 'KerasTensor' object has no attribute 'values'

In [19]:
from keras.models import Model

In [20]:
# Construct feature maps and reshape them into 256*256*3 for UNet
model = Model(inputs=feat_model.inputs, outputs=feat_model.layers[83].output)


layer_outputs = [layer.output for layer in model.layers]
feature_model = Model(inputs=model.input, outputs=layer_outputs)
feature_map_total = list()

NameError: name 'feat_model' is not defined

In [ ]:
# Generate batch 
iter_train = train_data.generator(num_neg=300, num_pos=300, data_augm=False,
                  color_normalization_file=color_normalization_file,
                  green_layer_only=False)
# Use the generator
i = 0
X_train, y_train = list(), list()

In [ ]:
for x, y in iter_train:
    X_train.append(x)
    y_train.append(y)
    i+=1
    if i > 0:
        break

In [ ]:
X_train = X_train[0]
y_train = y_train[0]

In [ ]:
for x in range(0,len(X_train)):
    feature_map = feature_model.predict(X_train[x].reshape(1,256,256,3)) 
    feature_map_total.append(feature_map)